<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Plugins for Decision Communication   

Decision Intelligence applied in this module:  
* Decision Scenario: Apply the Minto Pyramid framework to make Decision Communication systematic, efficient and clear.  
* Decision Scenario: Apply the Situation-Behavior-Impact-Recommendation (SBIR) framework to communicate decisions with potential negative feedback minimizing judgement.    

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets
* Use the API configuration to build the Semantic Kernel orchestrator
* The configuration builder will retrieve the Bing Search API Key

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.30.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.30.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.30.0-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Bing.WebSearch;
using System.ComponentModel;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var bingSearchAPIKey = config["BingSearch:APIKey"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0 Microsoft.SemanticKernel, 1.30.0 Microsoft.SemanticKernel.Plugins.Core, 1.30.0-alpha Microsoft.SemanticKernel.Plugins.Web, 1.30.0-alpha

In [2]:
Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Using Azure OpenAI Service


### Step 2 - Understanding & Explaining the Reasoning
> "The key to good decision making is not knowledge. It is understanding. We are swimming in the former. We are desperately lacking in the latter."
>
> -- <cite>Malcolm Gladwell (journalist, author, and public speaker known for his thought-provoking books)</cite> 

In the previous modules, Semantic Kernel with the Bing plugin and Auto Functions turned on where able to provide a seemingly "updated" answer. However, this answer can be improved further. In this next step, we will extract the understanging component from the GenAI LLM. Understanding refers to the ability to interpret, connect, and make sense of that knowledge. It's the ability to see the relationships between pieces of information, to recognize patterns, and to apply knowledge in context.

How can understanding be extracted from GenAI? **Instructing the GenAI system to communicate the decision or recommendation with a framework like the Minto Pyramid can provide an clear way illustrate the reasoning process to the user.** More information about the Minto Pyramid communication framework can be found on the Untools.co website: https://untools.co/minto-pyramid/    

<img width ="500px" src="https://assets-us-01.kc-usercontent.com/c6e42f10-0ed4-0062-585c-b740aa1ad46c/b3bbf2af-f6b2-4679-8a3a-837e37250990/minto-pyramid.png">

Execute the cell below to apply the Minto Pyramid Principle as guide for the GenAI model to structure the answer. Notice in the output the investigation of the decision is communicated in a clear & simple manner.  

In [3]:
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

#pragma warning disable SKEXP0050
var bingConnector = new BingConnector(bingSearchAPIKey);
var bingSearchPlugin = new WebSearchEnginePlugin(bingConnector);

semanticKernel.ImportPluginFromObject(bingSearchPlugin, "bing");
#pragma warning restore SKEXP0050

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    // Allows it to call the available Bing Plugin
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

// Prompt Template for Decision Iquiry
// Note the additional instruction to use the Minto Pyramid Principle
var promptTemplateString = """
What coaching decision did {{$basketBallTeam}} make after the 2024 playoffs? 
Only answer if you know the answer.
Use the Minto Pyramid Principle to structure your answer by 
starting with the conclusion, then provide key arguements and finally provide supporting details.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["basketBallTeam"] = "Phoenix Suns"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(promptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

**Conclusion:**
After the 2024 playoffs, the Phoenix Suns decided to fire head coach Frank Vogel.

**Key Arguments:**
1. **Disappointing Performance:** The Suns had high expectations for the season but were swept in the first round of the playoffs by the Minnesota Timberwolves.
2. **Short Tenure:** Frank Vogel was with the team for just one season, indicating that the management was not satisfied with his performance.
3. **Need for Change:** The decision to fire Vogel suggests that the Suns' management felt a change in leadership was necessary to meet their championship aspirations.

**Supporting Details:**
- The Phoenix Suns were eliminated in the first round of the 2024 NBA playoffs.
- The team had anticipated competing for the NBA championship but fell short of their goals.
- Frank Vogel's tenure as head coach lasted only one season, highlighting the urgency for a new direction.
- Following Vogel's dismissal, the Suns began searching for a new head coach to better align with their o

### Step 3 - Using a Communication Framework with minimal Judgement

There are many different communication frameworks that are avaialble for the Generative AI models to use. When communicating negative results and when that communication needs to be judgement-free, the "Situation-Behavior-Impact-Recommendation (SBIR)" framework is a great option. More information can be found on the Untool.co website: https://untools.co/situation-behavior-impact/.  

In this scenario, the decision of firing a coach is being investigated. It is a decision that has negative impact for various parties. Therefore, the SBIR framework is a great candidate for communicating this decision investigation. 

Execute the cell below to instruct the GenAI model to communicate the decision using SBIR. Contrast the output below with the Minto Pyramid output. From the two different outputs, one can see that the SBIR is feedback-oriented and action-driven, the Minto Pyramid Principle is designed for organizing and presenting complex information coherently.

In [4]:
// Prompt Template for Decision Iquiry
// Note the additional instruction to use the Minto Pyramid Principle
var promptTemplateString = """
What coaching decision did {{$basketBallTeam}} make after the 2024 playoffs? 
Only answer if you know the answer.
Use the Situation-Behavior-Impact-Recommendation (SBIR) framework to structure your answer.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["basketBallTeam"] = "Phoenix Suns"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(promptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

**Situation:** After the 2024 NBA playoffs, the Phoenix Suns faced a disappointing outcome as they were swept by the Minnesota Timberwolves in the first round.

**Behavior:** In response to this underperformance, the Phoenix Suns decided to fire their head coach, Frank Vogel, after just one season with the team.

**Impact:** This decision left the Suns with a head-coaching vacancy and highlighted the team's need for a new direction and leadership to improve their performance in future seasons.

**Recommendation:** It is recommended that the Phoenix Suns carefully evaluate potential candidates for the head coach position, considering their track record, coaching philosophy, and ability to lead the team to success in the highly competitive NBA environment.